In [137]:
import polars as pl
from pathlib import Path

import re


def test_unique_join(files_df: pl.DataFrame, database_df: pl.DataFrame):
    return files_df.join(database_df, on=["date"]).group_by("file_stem").len().sort("len", descending=True)


files_dir = Path("2200 Talks per directory")
database_df = pl.read_excel(
    "Progress - for Lohan.xlsm.xlsx"
)
database_df = database_df.drop_nulls("Date").with_columns(pl.col("Date").dt.strftime("%Y-%m-%d").alias("date")).filter(pl.col("Summary").str.to_lowercase().str.contains_any(["poem", "page", "extract","bhajans", "concert", "evening program", "letter", "quote", "arriv"]).not_()).with_columns(pl.col("Summary").str.slice(11).alias("Summary"))

database_with_date_df = database_df.filter(pl.col("date")!= "1970-01-01")

files = list(files_dir.rglob("*.docx"))
files_df = pl.DataFrame({"file_stem": [f.stem for f in files]})

files_df = files_df.with_columns(pl.col("file_stem").str.extract(r"^(\d{4}-\d{2}-\d{2})").alias("date"))
files_df = files_df.with_columns(pl.col("file_stem").str.replace(r"^(\d{4}-\d{2}-\d{2})", "").str.strip_chars(" –-").alias("Title"))

In [138]:
# separate files without date
files_with_date_df = files_df.filter(pl.col("date")!= "1970-01-01")

# Files without date

In [139]:
files_without_date_df = files_df.filter(pl.col("date")== "1970-01-01")
# files_without_date_df.write_excel("to_be_matched_manually.xlsx")

# Step 1: Match files and database with unique dates
Almost sure matches

In [ ]:
files_with_unique_date_df = files_with_date_df.filter(pl.col("date").len().over("date") == 1)
database_with_unique_date_df = database_with_date_df.filter(pl.col("date").len().over("date") == 1)

test_unique_join(files_with_unique_date_df, database_with_unique_date_df).item(0, "len")==1

In [141]:
all_matches = matched_step_1_df = files_with_unique_date_df.join(database_with_unique_date_df, on="date", suffix="_amruta").with_columns(pl.lit("1_unique_dates").alias("match_step"))
# joined_df.write_excel("almost_sure_matches.xlsx")

# Step 2: Match by date and city


In [ ]:
non_matched_in_step_1_df = files_with_date_df.filter(pl.col("file_stem").is_in(matched_step_1_df['file_stem']).not_())

print(f"Nothing wrong happened so far? {len(non_matched_in_step_1_df) + len(matched_step_1_df) == len(files_with_date_df)}")

print(f"Yet to be matched: {non_matched_in_step_1_df['file_stem'].n_unique()}")

In [143]:
non_matched_in_unique_date_partial_match_df = non_matched_in_step_1_df.join(database_with_date_df, on="date", suffix="_amruta")

matched_step_2_df = non_matched_in_unique_date_partial_match_df.filter(pl.col("file_stem").str.contains(pl.col("City"))).filter(pl.col("file_stem").len().over("file_stem")==1, pl.col("Amruta").len().over("Amruta") ==1).with_columns(pl.lit("2_date_and_city").alias("match_step"))

In [144]:
all_matches = pl.concat([all_matches, matched_step_2_df], how="diagonal")

database_minus_step_2 = database_with_date_df.filter(pl.col("Amruta").is_in(all_matches['Amruta']).not_())



# Step 3: Match data and [:5] string

In [145]:
non_matched_step_2_df = non_matched_in_step_1_df.filter(pl.col("file_stem").is_in(matched_step_2_df['file_stem']).not_())

In [146]:
matched_step_3_df = non_matched_step_2_df.join(database_minus_step_2, on="date", suffix="_amruta").filter(pl.col("Title").str.replace_all("[^A-Za-z]", "").str.contains(pl.col("Title_amruta").str.replace_all("[^A-Za-z]", "").str.slice(0,5))).filter(pl.col("file_stem").len().over("file_stem")==1).with_columns(pl.lit("3_date_and_string").alias("match_step"))

In [147]:
all_matches = pl.concat([all_matches, matched_step_3_df])

In [148]:
database_minus_step_3 = database_minus_step_2.filter(pl.col("Amruta").is_in(all_matches['Amruta']).not_())

# Step 4: More unique dates

In [ ]:
non_matched_step_3_df = non_matched_step_2_df.filter(pl.col("file_stem").is_in(matched_step_3_df['file_stem']).not_())
print(len(non_matched_step_3_df))

In [150]:
matched_step_4_df = non_matched_step_3_df.join(database_minus_step_3, on="date", suffix="_amruta").filter(pl.col("file_stem").len().over("file_stem")==1).with_columns(pl.lit("4_more_unique_dates").alias("match_step"))

In [151]:
all_matches = pl.concat([all_matches, matched_step_4_df])

In [152]:
database_minus_step_4 = database_minus_step_3.filter(pl.col("Amruta").is_in(all_matches['Amruta']).not_())


# Step 5: Tf-IDf similarity

In [ ]:
non_matched_step_4_df = non_matched_step_3_df.filter(pl.col("file_stem").is_in(matched_step_4_df['file_stem']).not_())

print(len(non_matched_step_4_df))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Get the titles from the non-matched dataframe
titles = non_matched_step_4_df['Title'].to_list()

# Fit and transform the titles to TF-IDF vectors
titles_vectorized = tfidf_vectorizer.fit_transform(titles)

# Print the shape of the resulting matrix
print(f"TF-IDF matrix shape: {titles_vectorized.shape}")

# If you want to see the feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Number of features (unique words): {len(feature_names)}")
print(f"Sample features: {feature_names[:10]}")

# Transform the database titles to TF-IDF vectors
database_vectorized = tfidf_vectorizer.transform(database_minus_step_4['Title'])
# Calculate similarity matrix using dot product between TF-IDF vectors


# Matrix multiplication of tfidf_matrix with the transpose of database_vectorized
# This gives us a similarity score between each title in non_matched_step_4_df and each title in database_minus_step_4
similarity_matrix = titles_vectorized.dot(database_vectorized.transpose())

# Print the shape of the similarity matrix
print(f"Similarity matrix shape: {similarity_matrix.shape}")
# The shape should be (n_non_matched_titles, n_database_titles)

# Optional: Convert to dense array for easier inspection of a sample
similarity_sample = similarity_matrix[:5, :5].toarray()
print("Sample of similarity matrix (first 5x5):")
print(similarity_sample)

In [155]:
joined_step_5_df = non_matched_step_4_df.with_row_index().join(database_minus_step_4.with_row_index(), on="date", suffix="_amruta")

In [156]:
def get_similarity_score(index, index_amruta, similarity_matrix=similarity_matrix):
    return similarity_matrix[index, index_amruta]

joined_step_5_df = joined_step_5_df.with_columns(pl.struct("index", "index_amruta").map_elements(lambda x: get_similarity_score(**x), return_dtype=pl.Float64).alias("similarity_score"))

In [ ]:
joined_step_5_df.group_by("file_stem").agg(pl.col("similarity_score").max().alias("max_score"), pl.col("similarity_score").eq(0.).not_().sum().alias("n_non_zero_scores")).sort("n_non_zero_scores", descending=True)

# Everything together

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Get the titles from the non-matched dataframe
titles = files_df['Title'].to_list()
amruta_titles = database_df['Title'].to_list()
summaries = database_df['Summary'].to_list()
# Fit and transform the titles to TF-IDF vectors
tfidf_vectorizer.fit(titles+amruta_titles+summaries)

titles_vectorized = tfidf_vectorizer.transform(titles)
amruta_titles_vectorized = tfidf_vectorizer.transform(amruta_titles)
summary_vectorized = tfidf_vectorizer.transform(summaries)
# If you want to see the feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Number of features (unique words): {len(feature_names)}")
print(f"Sample features: {feature_names[:10]}")


# Matrix multiplication of tfidf_matrix with the transpose of database_vectorized
# This gives us a similarity score between each title in non_matched_step_4_df and each title in database_minus_step_4
similarity_matrix = titles_vectorized.dot(amruta_titles_vectorized.transpose())
similarity_matrix_summary = titles_vectorized.dot(summary_vectorized.transpose())
# Print the shape of the similarity matrix
print(f"Similarity matrix shape: {similarity_matrix.shape}")
# The shape should be (n_non_matched_titles, n_database_titles)


In [ ]:
from rank_bm25 import BM25Okapi
import numpy as np
from scipy.sparse import csr_matrix

# Get the titles from the dataframes
titles = files_df['Title'].to_list()
amruta_titles = database_df['Title'].to_list()
summaries = database_df['Summary'].to_list()

# Tokenize the documents
tokenized_titles = [doc.lower().split() for doc in titles]
tokenized_amruta_titles = [doc.lower().split() for doc in amruta_titles]
tokenized_summaries = [doc.lower().split() for doc in summaries]

# Create BM25 models
bm25_titles = BM25Okapi(tokenized_amruta_titles)
bm25_summaries = BM25Okapi(tokenized_summaries)

# Create similarity matrices
similarity_matrix = np.zeros((len(titles), len(amruta_titles)))
similarity_matrix_summary = np.zeros((len(titles), len(summaries)))

# Calculate BM25 scores for each query (title) against all documents
for i, query in enumerate(tokenized_titles):
    title_scores = bm25_titles.get_scores(query)
    summary_scores = bm25_summaries.get_scores(query)
    similarity_matrix[i] = title_scores
    similarity_matrix_summary[i] = summary_scores

# Convert to sparse matrices for compatibility with existing code
similarity_matrix = csr_matrix(similarity_matrix)
similarity_matrix_summary = csr_matrix(similarity_matrix_summary)

# Get unique terms for feature names (similar to TF-IDF feature names)
all_tokens = set()
for doc in tokenized_titles + tokenized_amruta_titles + tokenized_summaries:
    all_tokens.update(doc)
feature_names = np.array(list(all_tokens))

print(f"Number of features (unique words): {len(feature_names)}")
print(f"Sample features: {feature_names[:10]}")

# Print the shape of the similarity matrix
print(f"Similarity matrix shape: {similarity_matrix.shape}")
# The shape should be (n_non_matched_titles, n_database_titles)


In [195]:
def get_similarity_score(index, index_amruta, similarity_matrix=similarity_matrix):
    return {"similarity_score": similarity_matrix[index, index_amruta]}



joined = files_df.with_row_index().join(database_df.with_row_index(), on="date", suffix="_amruta").with_columns(pl.struct("index", "index_amruta").map_elements(lambda x: get_similarity_score(**x), return_dtype=pl.Struct({"similarity_score": pl.Float64, "similarity_score_summary": pl.Float64})).alias("similarity_score"),
                                                                                                                pl.col("file_stem").str.contains(pl.col("City")).alias("city_match"),
                                                                                                                pl.col("Title").str.replace_all("[^A-Za-z]", "").str.contains(pl.col("Title_amruta").str.replace_all("[^A-Za-z]", "").str.slice(0,5)).alias("title_match"),
).sort(["file_stem", "similarity_score"], descending=True).unnest("similarity_score")

In [196]:
matches = joined.filter(pl.any_horizontal(pl.col("similarity_score")>0.1, 
                                pl.col("city_match"), pl.col("title_match")
                                )
              ).with_columns(pl.col("similarity_score").sort(descending=True).over("file_stem").alias("best_match")).drop("best_match", "index", "index_amruta")

In [ ]:
matches.sort(pl.col("file_stem").len().over("file_stem"), descending=True).drop("date").write_excel("matches.xlsx", autofit=True, table_style="Table Style Light 1")

In [ ]:
_matches = pl.read_excel("_matches.xlsx")
manually_matched_1 = _matches.filter(pl.col("is_match") == "1")
# manually_matched_1.write_excel("matches_1.xlsx", autofit=True, table_style="Table Style Light 1")

In [ ]:
a=matches.filter(pl.col("Amruta").is_in(manually_matched_1["Amruta"], ).not_(),pl.col("file_stem").is_in(manually_matched_1["file_stem"]).not_()).sort(pl.col("file_stem").len().over("file_stem"), pl.col("Date").dt.date(),"Title", "similarity_score", descending=True).drop("date")
a.write_excel("to_be_matched_manually_2.xlsx", autofit=True, table_style="Table Style Light 2")

In [ ]:
manually_matched_2 = pl.read_excel("_to_be_matched_manually_2.xlsx").filter(pl.col("is_match") == "1")

In [ ]:
manually_matched_by_llohann = pl.concat([manually_matched_1, manually_matched_2], how="diagonal")
manually_matched_by_llohann.write_excel("manually_matched_by_llohann.xlsx", autofit=True, table_style="Table Style Light 2")

In [ ]:
files_df.filter(pl.col("file_stem").is_in(manually_matched_by_llohann["file_stem"]).not_()).write_excel("files_to_be_matched_manually.xlsx", autofit=True, table_style="Table Style Light 2")

In [ ]:
manually_matched_2